In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
import src.cleanfun as cf

In [2]:
#cargamos csv
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
sharks.shape

(25723, 24)

In [3]:
sharks.sample(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
15252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2699,1989.01.08,08-Jan-1989,1989.0,Unprovoked,USA,Hawaii,"Wailua, Kaua'i",Swimming in strong current with 3 others when ...,Ken Ahlstrand,M,...,NaN,"J. Borg, p.77; L. Taylor (1993), pp.106-107",1989.01.08-Ahlstrand.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1989.01.08,1989.01.08,3604.0,NaN,NaN
10338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sharks.Type.value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [9]:
#eliminamos todas las columnas y filas que todos sus valores sean nulos
sharks.dropna(how="all", inplace=True)
sharks.dropna(how="all",axis = 1, inplace=True)

# Hipótesis
- hipotesis 1 los ataques de tiburones han icrmentado según han ido avanzando los años
- hipótesis 2 los ataquesde tiburones atacan suelen atacar a embarcaciones
- [hipóstesis 3 Florida es la capital mundial de los ataques de tiburones](https://www.lavanguardia.com/ocio/viajes/20210407/6631447/6-playas-mas-peligrosas-mundo.html)
- [hipótesis 4 ¿Es posible el ataque de tiburón en la costa española?](https://www.mundo-geo.es/naturaleza/es-posible-ataque-tiburon-en-costa-espanola_238643_102.html)

In [ ]:
#compruebo qué colmunas tienen datos interesantes para mis hipótesis y borro las que no me hacen ninguna falta.
sharks.columns

In [ ]:
columnas_no = ['Case Number','Investigator or Source','pdf', 'href formula',  'href',  'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23'] 
sharks.drop(columnas_no, axis=1, inplace=True)

In [ ]:
#ahora que hemos borrado columnas volvemos a hacer una pasada a los registros cuyos datos son todos nulos y borramos estas filas
sharks.dropna(how="all", inplace=True)

In [ ]:
#cambio nombre de algunas columnas que tienen espacios o caracteres especiales en el nombre
#"Sex " "Species " "Fatal (Y/N)"
cols = ["Sex ","Species ","Fatal (Y/N)"]
new_names = {"Sex " : "Sex",
                 "Species " : "Species",
                 "Fatal (Y/N)" : "Fatal"}
sharks.rename(columns = new_names, inplace=True)

In [ ]:
#segundo repaso a datos para ver si borramos alguna columna más que no me de información necesaria para las hipótesis
sharks.sample(20)
sharks.drop(["Name"],axis=1, inplace=True)

### Limpieza de datos nulos

In [ ]:
#miramos cuantos datos nulos hay en cda columna.
sharks.isna().sum()

#### rellenamos los nulos en función de los datos que contienen las columnas


In [ ]:
#Year
sharks.Year.unique()
#rellenamos años vacíos con 0
sharks.Year.fillna(0.0,inplace= True)
#convertirmos tipo de dato (float) a int.
sharks.Year = sharks.Year.astype(dtype = "int64")

In [ ]:
#type
sharks.Type.unique() #['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable','Sea Disaster', nan, 'Boat', 'Boatomg']
sharks[sharks["Type"] == "Invalid"].sample(20)
#invalid NO nos sirve para los nulos porque es para ataques que no fueron de tiburón
sharks.Type.fillna("UNKNOWN",inplace= True)
sharks.Type = sharks.Type.str.strip() #borramos espacios
sharks = sharks[sharks["Type"] != "Invalid"] #eliminamos los registros que que equivalen a "Invalid" ya que no son ataques de tibruón
#nos da mucha información si el ataque fue en embarcación o direcgtamente a "nadador"

In [ ]:
#Country, Area y Location -> nans == UNKNOWN
sharks["Country"].fillna("UNKNOWN", inplace = True)
sharks["Area"].fillna("UNKNOWN", inplace = True)
sharks["Location"].fillna("UNKNOWN", inplace = True)
sharks.Country = sharks.Country.str.strip()
sharks.Area = sharks.Area.str.strip()
sharks.Location = sharks.Location.str.strip()

In [ ]:
#activity
list(sharks.Activity.unique())
#de esta columna podemos sacar datos sobre si el tiburón atacó a embarcación o persona.
sharks["Activity"].fillna("UNKNOWN", inplace = True)
sharks.Activity = sharks.Activity.str.strip()

In [ ]:
#sex
sharks.Sex.unique()
#si el sexo de la víctima es desconocido lo rellenamos con "X"
#además queremos que se queden tres datos únicos -> M,F,X
sharks.Sex.fillna("X",inplace = True)
sharks.Sex = sharks.Sex.str.strip() #borramos espacios vacíos antes y después de cada cadena.
sharks.Sex.unique() #['F', 'M', 'X', 'lli', 'N', '.'] > lli, N y . == X
sharks["Sex"].replace('lli',"X",inplace=True)
sharks["Sex"].replace('.',"X",inplace=True)
sharks["Sex"].replace('N',"X",inplace=True)

In [ ]:
#Age
sharks.Age.unique()
#rellenamos nans con "UNKNOWN" porque 0 puede confundir la media si luego necesitamos usarla.
sharks.Age.fillna("UNKNOWN",inplace= True)
sharks.Age = sharks.Age.str.strip()

In [ ]:
#Injury > nan = UNKNOWN
sharks.Injury.unique()
sharks["Injury"].fillna("UNKNOWN", inplace = True)

In [ ]:
#Fatal
sharks.Fatal.unique
#nulos = UNKNOWN
sharks["Fatal"].fillna("UNKNOWN", inplace = True)
sharks.Fatal = sharks.Fatal.str.strip()
sharks.Fatal.unique() #comprobar registros con FAtal == 'M', '2017', 'y'
sharks[(sharks["Fatal"] == 'M')|(sharks["Fatal"] == "2017")|(sharks["Fatal"] == "y")].Injury.unique
sharks["Fatal"].replace('M',"N",inplace=True)
sharks["Fatal"].replace('y',"Y",inplace=True)
sharks["Fatal"].replace('2017',"N",inplace=True)
sharks.Fatal.unique() #['N', 'Y', 'UNKNOWN']

In [ ]:
#Time
sharks.Time.unique()
#nulos = UNKNOWN
sharks["Time"].fillna("UNKNOWN", inplace = True)
#puede que esta columna no la utlicemos para ningún análisis. No nos interesan los datos tan exactos de la hora del ataque.

In [ ]:
#Species
list(sharks.Species.unique())
#nulos = UNKNOWN
sharks["Species"].fillna("UNKNOWN", inplace = True)
sharks.Species = sharks.Species.str.strip()

In [ ]:
sharks.isna().sum()

### Procesamos el texto de las columnas

##### Date, Year.

In [ ]:
sharks.Date.value_counts()
#de la columna Date extraemos años y meses para nuevas columnas
sharks["Year_D"] = sharks["Date"].str.extract(r'(\d{4})')
sharks["Month_D"] = sharks["Date"].str.extract(r'([A-Z][a-z][a-z]-\d{4})')
sharks["Month_D"] = sharks["Month_D"].str.extract(r'([A-Z][a-z][a-z])')
#volvemos a rellenar los nulos que quedan 
#sharks["Date"].fillna("UNKNOWN", inplace = True)
sharks.Year_D.fillna("UNKNOWN",inplace= True)
sharks.Month_D.fillna("UNKNOWN",inplace= True)
#convertimos los strings de Year_D enteros
sharks.Year_D = sharks.Year_D.apply(cf.entero_if)
sharks["Month_D"] = sharks["Month_D"].str.upper()

##### Type

In [ ]:
sharks.Type.unique()

In [ ]:
#Boating, Boat y Boatomg == mismo dato "Boat"
sharks["Type"] = sharks["Type"].str.replace((".*[Bb](OAT|oat).*"),"Boat", regex=True)
sharks.Type.unique()

#####  Activity

In [ ]:
#creamos una nueva columna "Act" strings que acaban en "ing" de "Activity"
sharks["Act"] = sharks.Activity.str.extract(r'(\w+ing)+')
#rellenamos los nuevos nans
sharks["Act"].fillna("UNKONWN", inplace = True)
sharks["Act"]= sharks["Act"].str.upper()

In [ ]:
sharks.Act.value_counts().head(10)

###### Activity -> "Fishing"
- como en Type tenemos "Boat" para saber si hay ataques direcatemente a los barcos, pero luego en activity "Fishing" la mayoría son desde barcos, vamos a crear una columna "Boat" incluyendo todos los registros que son type "Boat" además de los que no son "Boat" pero son Fishin

In [ ]:
#filtramos todos registros que son tipo Boat y de los que no son tipo Boat pero son activity fishing para ver cuantos realemnte van en barco
sharks["Boat"] = np.where(((sharks["Type"] == "Boat")|((sharks["Type"] != "Boat") & (sharks["Act"] == "FISHING"))),"Yes","No")

In [ ]:
sharks.Boat.value_counts()

#####  Sex

In [ ]:
sharks.Sex.unique() #no hacemos nada más

#####  Age

In [ ]:
#dejamos solos los valores de edades de Age:
sharks.Age = sharks.Age.apply(cf.edades)
sharks.Age.unique()

#####  Injury

In [ ]:
#creamos nueva columna con un resumen de tipos de lesiones
sharks["Injur_Type"] = sharks.Injury.apply(cf.actividades)
sharks["Injur_Type"] = sharks["Injur_Type"].str.upper()

In [ ]:
sharks.Injur_Type.unique()

##### Fatal

In [ ]:
sharks.Fatal.unique() #no hacemos nada más

##### Time
no vamos a usar esta columna, porque para nuestros datos no nos intersa a qué hora pasaron los incidentes

#####  species

In [ ]:
#extraemos en una nueva columna en la que extraemos las especies de los tiburones desde la columna "Species"
sharks["Spec"] = sharks.Species.str.extract(r'(\w{3,}(?= shark))')

In [ ]:
sharks["Spec"].fillna("UNKONWN", inplace = True)
sharks["Spec"] = sharks["Spec"].str.upper()

In [ ]:
sharks.Spec.value_counts().head(5)

#####  country // Areas

In [ ]:
sharks["Country"] = sharks["Country"].str.upper()
sharks["Area"] = sharks["Area"].str.upper()
sharks.Area.value_counts().head(10) #las areas nos interesan sobre todo Florida en comparación con el resto...

## Exportación

In [ ]:
sharks.sample(5)

In [ ]:
#exportarmos sharky
sharks.to_csv("data/sharks.csv")